In [1]:
import requests

import pandas as pd

from bs4 import BeautifulSoup

import time

pd.set_option('display.max_columns', 50)

In [2]:
years = list(range(2024,2021, -1))
years

[2024, 2023, 2022]

In [3]:
connect_timeout = 6
read_timeout = 100

def stats(link, year):
     
    data = requests.get(link, timeout=(connect_timeout, read_timeout))
    soup = BeautifulSoup(data.text)
    
    stats_table = pd.read_html(data.text, match="Regular season Table")[0]
    
    standard_stats = pd.read_html(data.text, match = "Squad Standard Stats")[0]
    standard_stats.columns = standard_stats.columns.droplevel()
    
    goalkeeping_stats = pd.read_html(data.text, match="Squad Goalkeeping")[0]
    goalkeeping_stats.columns = goalkeeping_stats.columns.droplevel()
    
    shooting_stats = pd.read_html(data.text, match="Squad Shooting")[0]
    shooting_stats.columns = shooting_stats.columns.droplevel()
    
    stats_standard = stats_table.merge(standard_stats[["Squad", "Poss", "Gls", "Ast"]], on="Squad", how="left")
    stats_standard_gk = stats_standard.merge(goalkeeping_stats[["Squad", "CS%"]], on="Squad", how="left")
    all_stats = stats_standard_gk.merge(shooting_stats[["Squad", "SoT/90", "G/SoT", "SoT%"]], on="Squad", how="left")
        
    all_stats["Season"] = year
    
    return all_stats
    

In [4]:
overall_stats = []

In [12]:
def concate():
    
    all_stats_2024 = stats("https://fbref.com/en/comps/9/Premier-League-Stats", 2024)
    all_stats_2024 = all_stats_2024.drop(["Last 5"], axis=1)
    all_stats_2023 = stats("https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats", 2023)
    all_stats_2022 = stats("https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats", 2022)
    all_stats_2021 = stats("https://fbref.com/en/comps/9/2020-2021/2020-2021-Premier-League-Stats", 2021)
    all_stats_2020 = stats("https://fbref.com/en/comps/9/2019-2020/2019-2020-Premier-League-Stats", 2020)
    all_stats_2019 = stats("https://fbref.com/en/comps/9/2018-2019/2018-2019-Premier-League-Stats", 2019)
    
    overall_stats.append(all_stats_2024)
    overall_stats.append(all_stats_2023)
    overall_stats.append(all_stats_2022)
    overall_stats.append(all_stats_2021)
    overall_stats.append(all_stats_2020)
    overall_stats.append(all_stats_2019)
    
    overall_stats_df = pd.concat(overall_stats)
    return overall_stats_df
    

In [13]:
overall_stats_df = concate()

In [14]:
overall_stats_df.head()

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,Notes,Poss,Gls,Gls,Ast,Ast,CS%,SoT/90,G/SoT,SoT%,Season
0,1,Liverpool,21,14,6,1,47,18,29,48,2.29,45.7,24.0,21.7,1.03,52619,Mohamed Salah - 14,Alisson,NaN,60.2,44,2.10,34,1.62,38.1,6.19,0.31,33.5,2024
1,2,Manchester City,20,13,4,3,48,23,25,43,2.15,39.7,19.1,20.6,1.03,53297,Erling Haaland - 14,Ederson,NaN,64.1,47,2.35,35,1.75,25.0,6.50,0.33,39.0,2024
2,3,Arsenal,21,13,4,4,42,20,22,43,2.05,39.4,16.7,22.7,1.08,60180,Bukayo Saka - 6,David Raya,NaN,60.3,41,1.95,27,1.29,38.1,5.00,0.33,30.9,2024
3,4,Aston Villa,21,13,4,4,43,27,16,43,2.05,37.2,26.7,10.4,0.50,41495,Ollie Watkins - 9,Emiliano Martínez,NaN,55.2,40,1.90,30,1.43,23.8,5.05,0.34,34.8,2024
4,5,Tottenham,21,12,4,5,44,31,13,40,1.90,36.4,35.5,0.9,0.04,61624,Son Heung-min - 12,Guglielmo Vicario,NaN,59.3,41,1.95,36,1.71,23.8,5.81,0.33,36.4,2024


In [15]:
overall_stats_df.shape

(180, 29)

In [16]:
overall_stats_df.to_csv("datasets/overall_stats.csv", index=False)